# **_Event Extraction_**

`Import Packeges`  
Using spacy dependency parser model

In [1]:
import spacy
from spacy import displacy  # Use displacy to visualize the dependency tree
import re
import nltk
import warnings

nlp = spacy.load('en_core_web_trf')  # load language model
warnings.filterwarnings("ignore")
# ignore warning, mainly ignore the warning about GPU

`Load Data`  
We load the corpus to a list

In [2]:
corpus = []
with open("./data/dataset.txt", mode='r') as file:
    sentence = file.readline()
    while sentence:
        corpus += [sentence]
        sentence = file.readline()


`Preprocess Corpus`  
We mainly use regular expression to process the corpus

In [3]:
def get_trigger(sentence: str):
    trigger = []
    tags = [' <e> ', ' </e> ']
    for tag in tags:
        lst = sentence.split(tag)
        for i in range(0, len(lst) - 1):
            trigger.append(lst[i].split(' ')[-1])
    return trigger


triggers = [get_trigger(sentence) for sentence in corpus]
corpus = [re.sub(" <[\S]*> ", " ", s) for s in corpus]
corpus = [re.sub(" \. ", ".", s) for s in corpus]
corpus = [re.sub("[\.]{3}", " ... ", s) for s in corpus]
corpus = [re.sub(" , ", ", ", s) for s in corpus]
corpus = [re.sub(" - ", " ", s) for s in corpus]
corpus = [re.sub("[ ]{2,}", " ", s) for s in corpus]
corpus = [re.sub("\n", "", s) for s in corpus]

`Get Dependency Tree`  

In [6]:
"""
The speed of dependency parsing is about 10 sentence per second.
"""
dependency = [nlp(sentence) for sentence in corpus[:10]]


def transform(sentence):
    def filter(word):
        lst = [',', ' ', '.', '"', "'"]
        for s in lst:
            if s == str(word):
                return False
        return True

    def to_nltk_tree(node):
        if node.n_lefts + node.n_rights > 0:
            return node, [
                to_nltk_tree(child) for child in node.children if filter(child)
            ]
        else:
            return node

    return [to_nltk_tree(sent.root) for sent in sentence.sents]


trees = [transform(sentence) for sentence in dependency]


`Extract Event`

In [ ]:
def find_movement(trigger):
    """
    This function find the root of the subtree discribing the event
    containing the trigger. 
    Note that the root is a verb, the following work will base on this subtree
    Args:
        trigger: event trigger words
    """
    pass


def find_subject(moment, trigger, exclude):
    """
    
    Args:
        moment (_type_): _description_
        trigger (_type_): _description_
        exclude (_type_): _description_
    """
    pass


def find_object(moment, trigger, exclude):
    """
    
    Args:
        moment (_type_): _description_
        trigger (_type_): _description_
        exclude (_type_): _description_
    """
    pass


` `

In [5]:
def extract(trigger):
    """
    Extract event and event attribute vai mention tokens generate by spacy
    Args:
    trigger: event trigger words
    Returns:
    """
    movement = find_movement(trigger)
    subject, passive = find_subject(movement, trigger, exclude=[movement])
    exclude = [movement]
    if isinstance(subject, list):
        exclude.extend(subject)
    else:
        exclude.append(subject)
    obj = find_object(movement, trigger, exclude=exclude)
    if passive or subject == obj:
        obj = None
    return (subject, passive), movement, obj